In [14]:
from sympy import *
import numpy as np

Txx, Tyy, Txy = symbols('Txx, Tyy, Txy ')
Exx,Eyy,Exy   = symbols('Exx,Eyy,Exy')
C_pwl, n_pwl  = symbols('C_pwl, n_pwl')
C_lin, n_lin, m_lin  = symbols('C_lin, n_lin, m_lin')
eta_0,eta_el  = symbols('eta_0,eta_el')
dg = symbols('dg') # grain size

# The new guys for anisotropy
d0, d1, ani, two, a = symbols('d0, d1, ani, two, a')
f_ani         = symbols('f_ani')

Tzz      = -Txx-Tyy 
Ezz      = -Exx-Eyy 
Tii       = sqrt(1/2*(Txx**2+Tyy**2+Tzz**2) + a**2*Txy**2)
Jii           = Tii**2
Eii       = sqrt(1/2*(Exx**2+Eyy**2+Ezz**2) + a**2*Exy**2)

display(Tii.diff(Txx).subs(Tii,'Tii'))
display(Tii.diff(Tyy).subs(Tii,'Tii'))
display(Tii.diff(Txy).subs(Tii,'Tii'))


# Gzz = symbols('Gzz')
C_ANI    = Matrix([ [-d0, d0, two*d1], [d0, -d0, -two*d1], [d1, -d1, -two*(1/2-d0)]]) ##### !!! - sign in D33 -a0
C_ISO    = Matrix([ [1, 0, 0], [0, 1, 0], [0, 0, two*1/2]])
Dani     = 2* (C_ISO + C_ANI * ani)
E        = Matrix([[Exx],[Eyy],[Exy]])
Gij      = Matrix([ [1/2, 0, 0], [0, 1/2, 0], [0, 0, 1/two]]) * Dani*E
Gxx, Gyy, Gxy = Gij[0], Gij[1], Gij[2]
Gzz      = -Gxx-Gyy 
Gii       = sqrt(1/2*(Gxx**2+Gyy**2+Gzz**2) + a**2*Gxy**2)


(Txx + 0.5*Tyy)/Tii

(0.5*Txx + Tyy)/Tii

Txy**two*a**2*two/(2*Tii*Txy)

In [2]:

#---------------------- VISCO-ELASTIC VISCOSITY ----------------------#

# 1. Viscosity expression as function of stress invariant
eta_pwl       = f_ani**(n_pwl-1)*(2*C_pwl)**(-1)*Tii**(1-n_pwl)
eta_lin       = f_ani**(n_lin-1)*(2*C_lin)**(-1)*Tii**(1-n_lin) * dg**(m_lin)
eta_ve  = (eta_el**-1 + eta_pwl**-1 + eta_lin**-1)**-1
eta_ve  = (eta_el**-1 + eta_pwl**-1 )**-1
print('eta_ve = ' + octave_code(eta_ve.subs(eta_pwl,'eta_pwl').subs(f_ani,'f_ani')) + ';')
# 2. Derivatives of viscosity w.r.t. strain rate components
detadexx = symbols('detadexx')
detadtxx = symbols('detadtxx')
f        = detadexx - ( detadtxx * (2*eta_0 + 2*detadexx*Exx))
detadexx = solve(f,detadexx)[0]
display(detadexx)
print('detadexx = ' + octave_code(detadexx.subs(1/2*Txx**2 + Txy**2*a**2 + 1/2*Tyy**2 + 1/2*Tzz**2,'Jii').subs(f_ani,'f_ani')) + ';')
# 3. Derivatives of viscosity w.r.t. stress components
detadtxx = eta_ve.diff(Txx)
detadtyy = eta_ve.diff(Tyy)
detadtxy = eta_ve.diff(Txy)
detadtxx = detadtxx.subs(eta_ve,'eta_ve').subs(eta_ve,'eta_lin').subs(eta_ve,'eta_pwl').subs(1/2*Txx**2 + Txy**2*a**2 + 1/2*Tyy**2 + 1/2*Tzz**2,'Jii').subs(f_ani.diff(Txx),'dfanidTxx').subs(f_ani,'f_ani').simplify()
detadtyy = detadtyy.subs(eta_ve,'eta_ve').subs(1/2*Txx**2 + Txy**2*a**2 + 1/2*Tyy**2 + 1/2*Tzz**2,'Jii').subs(f_ani.diff(Tyy),'dfanidTyy').subs(f_ani,'f_ani').simplify()
detadtxy = detadtxy.subs(eta_ve,'eta_ve').subs(1/2*Txx**2 + Txy**2*a**2 + 1/2*Tyy**2 + 1/2*Tzz**2,'Jii').subs(f_ani.diff(Txy),'dfanidTxy').subs(f_ani,'f_ani').simplify()

display(detadtxx)
display(detadtyy)
display(detadtxy)
# for i in range(1):
#     print('detadTxx = ' + ccode(detadtxx) + ';')
#     print('detadTyy = ' + ccode(detadtyy) + ';')
#     print('detadTxy = ' + ccode(detadtxy) + ';')
for i in range(1):
    print('detadTxx = ' + octave_code(detadtxx) + ';')
    print('detadTyy = ' + octave_code(detadtyy) + ';')
    print('detadTxy = ' + octave_code(detadtxy) + ';')


eta_ve = 1./(2*C_pwl.*f_ani.^(1 - n_pwl).*(0.5*Txx.^2 + Txy.^2.*a.^2 + 0.5*Tyy.^2 + 0.5*(-Txx - Tyy).^2).^(n_pwl/2 - 1/2) + 1./eta_el);


-2*detadtxx*eta_0/(2*Exx*detadtxx - 1)

detadexx = -2*detadtxx.*eta_0./(2*Exx.*detadtxx - 1);


-C_pwl*Jii**(n_pwl/2 - 3/2)*eta_ve**2*f_ani**(1 - n_pwl)*(2.0*Txx + 1.0*Tyy)*(n_pwl - 1)

-C_pwl*Jii**(n_pwl/2 - 3/2)*eta_ve**2*f_ani**(1 - n_pwl)*(1.0*Txx + 2.0*Tyy)*(n_pwl - 1)

-2*C_pwl*Jii**(n_pwl/2 - 3/2)*Txy*a**2*eta_ve**2*f_ani**(1 - n_pwl)*(n_pwl - 1)

detadTxx = -C_pwl.*Jii.^(n_pwl/2 - 3/2).*eta_ve.^2.*f_ani.^(1 - n_pwl).*(2.0*Txx + 1.0*Tyy).*(n_pwl - 1);
detadTyy = -C_pwl.*Jii.^(n_pwl/2 - 3/2).*eta_ve.^2.*f_ani.^(1 - n_pwl).*(1.0*Txx + 2.0*Tyy).*(n_pwl - 1);
detadTxy = -2*C_pwl.*Jii.^(n_pwl/2 - 3/2).*Txy.*a.^2.*eta_ve.^2.*f_ani.^(1 - n_pwl).*(n_pwl - 1);


In [10]:
# ## !!!!!!!!!! THIS IS NOT NEEDED AT ALL
# f_ani     = Gii/Eii
# dfanidExx = f_ani.diff(Exx).subs(f_ani, 'f_ani').subs(Eii, 'Eii').subs(Gii, 'Gii').subs(Gxx,'Gxx').subs(Gyy,'Gyy').subs(Gxy,'Gxy').subs(Gzz,'Gzz')
# dfanidEyy = f_ani.diff(Eyy).subs(f_ani, 'f_ani').subs(Eii, 'Eii').subs(Gii, 'Gii').subs(Gxx,'Gxx').subs(Gyy,'Gyy').subs(Gxy,'Gxy').subs(Gzz,'Gzz')
# dfanidExy = f_ani.diff(Exy).subs(f_ani, 'f_ani').subs(Eii, 'Eii').subs(Gii, 'Gii').subs(Gxx,'Gxx').subs(Gyy,'Gyy').subs(Gxy,'Gxy').subs(Gzz,'Gzz')

# # display(dfanidExx.subs('f_ani',1.0).subs(ani,0.0).subs(d0,0.0).subs(d1,1.0).subs('Gxx',Exx).subs('Gii','Eii'))
# print('dfanidExx = ' + octave_code(dfanidExx) + ';')
# print('dfanidEyy = ' + octave_code(dfanidEyy) + ';')
# print('dfanidExy = ' + octave_code(dfanidExy) + ';')
# # display(f_ani.subs(Gii, 'Gii').subs(ani,0.0))


In [15]:
# For grain size evolution one needs to make the viscosity of linear creep stress dependent
df = Function('df')(Txx,Tyy,Txy) # grain size

# 1. Viscosity expression as function of stress invariant
eta_lin       = f_ani**(n_lin-1)*(2*C_lin)**(-1)*Tii**(1-n_lin) * df**(m_lin)
eta_ve  = (eta_el**-1 + eta_lin**-1)**-1
#eta_pwl       = f_ani**(n_pwl-1)*(2*C_pwl)**(-1)*Tii**(1-n_pwl)
#eta_ve  = (eta_el**-1 + eta_pwl**-1)**-1

# 3. Derivatives of viscosity w.r.t. stress components
detadtxx = eta_ve.diff(Txx)
detadtyy = eta_ve.diff(Tyy)
detadtxy = eta_ve.diff(Txy)
detadtxx = detadtxx.subs(eta_ve,'eta_ve').subs(eta_ve,'eta_lin').subs(1/2*Txx**2 + Txy**2*a**2 + 1/2*Tyy**2 + 1/2*Tzz**2,'Jii').subs(df.diff(Txx),'ddgdTxx').subs(df,'dg').simplify()
detadtyy = detadtyy.subs(eta_ve,'eta_ve').subs(eta_ve,'eta_lin').subs(1/2*Txx**2 + Txy**2*a**2 + 1/2*Tyy**2 + 1/2*Tzz**2,'Jii').subs(df.diff(Tyy),'ddgdTyy').subs(df,'dg').simplify()
detadtxy = detadtxy.subs(eta_ve,'eta_ve').subs(eta_ve,'eta_lin').subs(1/2*Txx**2 + Txy**2*a**2 + 1/2*Tyy**2 + 1/2*Tzz**2,'Jii').subs(df.diff(Txy),'ddgdTxy').subs(df,'dg').simplify()

display(detadtxx)

print('detadtxx = ' +  octave_code(detadtxx) + ';')
print('detadtyy = ' +  octave_code(detadtyy) + ';')
print('detadtxy = ' +  octave_code(detadtxy) + ';')

C_lin*dg**(-2*m_lin - 1)*eta_ve**2*f_ani**(1 - n_lin)*(2*ddgdTxx*dg**m_lin*m_lin*(0.5*Txx**2 + Txy**two*a**2 + 0.5*Tyy**2 + 0.5*(Txx + Tyy)**2)**(n_lin/2 + 1/2) - dg**(m_lin + 1)*(2.0*Txx + 1.0*Tyy)*(n_lin - 1)*(0.5*Txx**2 + Txy**two*a**2 + 0.5*Tyy**2 + 0.5*(Txx + Tyy)**2)**(n_lin/2 - 1/2))/(0.5*Txx**2 + Txy**two*a**2 + 0.5*Tyy**2 + 0.5*(Txx + Tyy)**2)

detadtxx = C_lin.*dg.^(-2*m_lin - 1).*eta_ve.^2.*f_ani.^(1 - n_lin).*(2*ddgdTxx.*dg.^m_lin.*m_lin.*(0.5*Txx.^2 + Txy.^two.*a.^2 + 0.5*Tyy.^2 + 0.5*(Txx + Tyy).^2).^(n_lin/2 + 1/2) - dg.^(m_lin + 1).*(2.0*Txx + 1.0*Tyy).*(n_lin - 1).*(0.5*Txx.^2 + Txy.^two.*a.^2 + 0.5*Tyy.^2 + 0.5*(Txx + Tyy).^2).^(n_lin/2 - 1/2))./(0.5*Txx.^2 + Txy.^two.*a.^2 + 0.5*Tyy.^2 + 0.5*(Txx + Tyy).^2);
detadtyy = C_lin.*dg.^(-2*m_lin - 1).*eta_ve.^2.*f_ani.^(1 - n_lin).*(2*ddgdTyy.*dg.^m_lin.*m_lin.*(0.5*Txx.^2 + Txy.^two.*a.^2 + 0.5*Tyy.^2 + 0.5*(Txx + Tyy).^2).^(n_lin/2 + 1/2) - dg.^(m_lin + 1).*(1.0*Txx + 2.0*Tyy).*(n_lin - 1).*(0.5*Txx.^2 + Txy.^two.*a.^2 + 0.5*Tyy.^2 + 0.5*(Txx + Tyy).^2).^(n_lin/2 - 1/2))./(0.5*Txx.^2 + Txy.^two.*a.^2 + 0.5*Tyy.^2 + 0.5*(Txx + Tyy).^2);
detadtxy = C_lin.*dg.^(-2*m_lin - 1).*eta_ve.^2.*f_ani.^(1 - n_lin).*(2*Txy.*ddgdTxy.*dg.^m_lin.*m_lin.*(0.5*Txx.^2 + Txy.^two.*a.^2 + 0.5*Tyy.^2 + 0.5*(Txx + Tyy).^2).^(n_lin/2 + 1/2) - Txy.^two.*a.^2.*dg.^(m_lin + 1).*two.*(n_lin - 1).*

In [16]:
# Evaluate the product T * Edis
Ared,dg0,dt,Agro,p = symbols('Ared,dg0,dt,Agro,p')
Tii = symbols('Tii')
Eii = symbols('Eii')
f_ani = symbols('f_ani')
Gxx,Gyy,Gzz,Gxy = symbols('Gxx,Gyy,Gzz,Gxy')
# Dev. stress
Txx       = Tii/f_ani/Eii*Gxx
Tyy       = Tii/f_ani/Eii*Gyy
Tzz       = -Txx-Tyy
Txy       = Tii/f_ani/Eii*Gxy
# Dis. Dev. strain
Eii_pwl   = f_ani**(-n_pwl) * C_pwl * Tii**(n_pwl);  
exx_pwl   = Exx/Gxx*f_ani * Txx/Tii * Eii_pwl
eyy_pwl   = Eyy/Gyy*f_ani * Tyy/Tii * Eii_pwl      
ezz_pwl   = -exx_pwl - eyy_pwl     
exy_pwl   = Exy/Gxy*f_ani * Txy/Tii * Eii_pwl 
# Work
Wpwl      = exx_pwl*Txx + eyy_pwl*Tyy + ezz_pwl*Tzz + two*exy_pwl*Txy
Wpwl_subs = Wpwl.subs(Eii_pwl,'Eii_pwl').simplify()
display(Wpwl_subs)
# print('Wred = ' + octave_code(Wpwl_subs) + ';')
# Sensitivity to Tii
# display(Wpwl)
dWreddTii = Wpwl.diff(Tii).subs(Wpwl,'Wred').subs(Eii_pwl,'Eii_pwl').simplify()
display(dWreddTii)
print('dWreddTii = ' + octave_code(dWreddTii) + ';')

Eii_pwl*Tii*(Exx*Gxx + Exy*Gxy*two + Eyy*Gyy + (Exx + Eyy)*(Gxx + Gyy))/(Eii**2*f_ani)

Eii_pwl*(Exx*Gxx*n_pwl + Exx*Gxx + Exy*Gxy*n_pwl*two + Exy*Gxy*two + Eyy*Gyy*n_pwl + Eyy*Gyy + n_pwl*(Exx + Eyy)*(Gxx + Gyy) + (Exx + Eyy)*(Gxx + Gyy))/(Eii**2*f_ani)

dWreddTii = Eii_pwl.*(Exx.*Gxx.*n_pwl + Exx.*Gxx + Exy.*Gxy.*n_pwl.*two + Exy.*Gxy.*two + Eyy.*Gyy.*n_pwl + Eyy.*Gyy + n_pwl.*(Exx + Eyy).*(Gxx + Gyy) + (Exx + Eyy).*(Gxx + Gyy))./(Eii.^2.*f_ani);


In [21]:
# Evaluate sensitivity of grain size to tau
dg1          = Function('dg1')(Tii)
Wpwl1        = Function('Wpwl1')(Tii)
ddgdTii      = symbols('ddgdTii')
dred         =-Wpwl1*dg1**2*Ared
dgro         = Agro*dg1**(1-p)
dg           = (dg0) + dt*(dred + dgro)
ddgdTii0     = dg.diff(Tii)
f            = ddgdTii - ddgdTii0.subs(dg1.diff(Tii), ddgdTii).subs(dg1,'dg')
display(f)
ddgdTii1     = solve(f,ddgdTii)[0]
ddgdTii_subs = ddgdTii1.subs(Wpwl1.diff(Tii),'dWreddTii').subs(Wpwl1,'Wred')
display(ddgdTii_subs)
print('ddgdTii = ' + octave_code(ddgdTii_subs) + ';')

ddgdTii - dt*(Agro*ddgdTii*dg**(1 - p)*(1 - p)/dg - 2*Ared*ddgdTii*dg*Wpwl1(Tii) - Ared*dg**2*Derivative(Wpwl1(Tii), Tii))

-Ared*dWreddTii*dg**2*dt/(Agro*dg**(-p)*dt*p - Agro*dg**(-p)*dt + 2*Ared*Wred*dg*dt + 1)

ddgdTii = -Ared.*dWreddTii.*dg.^2.*dt./(Agro.*dg.^(-p).*dt.*p - Agro.*dg.^(-p).*dt + 2*Ared.*Wred.*dg.*dt + 1);


In [18]:
# For grain size evolution one needs to make the viscosity of linear creep stress dependent
Tii = symbols('Tii')

# 1. Viscosity expression as function of stress invariant
eta_lin       = f_ani**(n_lin-1)*(2*C_lin)**(-1)*Tii**(1-n_lin) * dg1**(m_lin)
eta_pwl       = f_ani**(n_pwl-1)*(2*C_pwl)**(-1)*Tii**(1-n_pwl)
eta_ve        = (eta_el**-1 + eta_pwl**-1 + eta_lin**-1)**-1

# 3. Derivatives of viscosity w.r.t. stress components
detadTii = eta_ve.diff(Tii)

detadTii = detadTii.subs(eta_ve,'eta_ve').subs(eta_ve,'eta_lin').subs(dg1.diff(Tii),'ddgdTii').subs(dg1,'dg').simplify()

display(detadTii)

print('detadTii = ' +  octave_code(detadTii) + ';')


2*dg**(-2*m_lin - 1)*eta_ve**2*(-C_lin*Tii**n_lin*dg**(m_lin + 1)*f_ani**(1 - n_lin)*(n_lin - 1) + C_lin*Tii**(n_lin + 1)*ddgdTii*dg**m_lin*f_ani**(1 - n_lin)*m_lin - C_pwl*Tii**n_pwl*dg**(2*m_lin + 1)*f_ani**(1 - n_pwl)*(n_pwl - 1))/Tii**2

detadTii = 2*dg.^(-2*m_lin - 1).*eta_ve.^2.*(-C_lin.*Tii.^n_lin.*dg.^(m_lin + 1).*f_ani.^(1 - n_lin).*(n_lin - 1) + C_lin.*Tii.^(n_lin + 1).*ddgdTii.*dg.^m_lin.*f_ani.^(1 - n_lin).*m_lin - C_pwl.*Tii.^n_pwl.*dg.^(2*m_lin + 1).*f_ani.^(1 - n_pwl).*(n_pwl - 1))./Tii.^2;


In [41]:
# For local Jacobian
Ared,dg0,dt,Agro,p,dg = symbols('Ared,dg0,dt,Agro,p,dg')
Tii = symbols('Tii')
Eii = symbols('Eii')
f_ani = symbols('f_ani')
Gxx,Gyy,Gzz,Gxy = symbols('Gxx,Gyy,Gzz,Gxy')
# Dev. stress
Txx       = Tii/f_ani/Eii*Gxx
Tyy       = Tii/f_ani/Eii*Gyy
Tzz       = -Txx-Tyy
Txy       = Tii/f_ani/Eii*Gxy
# Dis. Dev. strain
Eii_pwl   = f_ani**(-n_pwl) * C_pwl * Tii**(n_pwl);  
exx_pwl   = Exx/Gxx*f_ani * Txx/Tii * Eii_pwl
eyy_pwl   = Eyy/Gyy*f_ani * Tyy/Tii * Eii_pwl      
ezz_pwl   = -exx_pwl - eyy_pwl     
exy_pwl   = Exy/Gxy*f_ani * Txy/Tii * Eii_pwl 
# Work
Wpwl      = exx_pwl*Txx + eyy_pwl*Tyy + ezz_pwl*Tzz + two*exy_pwl*Txy
dred         =-Wpwl*dg**2*Ared
dgro         = Agro*dg**(1-p)
fdg          = dg - (dg0 + dt*(dred + dgro) )

display(fdg.diff(dg).subs(fdg,'fdg').subs(dgro,'dgro').subs(dred,'dred').subs(Wpwl,'Wred'))
display(fdg.diff(Tii).subs(fdg,'fdg').subs(dgro,'dgro').subs(dred,'dred').subs(Wpwl,'Wred').subs(exy_pwl,'exy_pwl').subs(exx_pwl,'exx_pwl').subs(eyy_pwl,'eyy_pwl').subs(Eii_pwl,'Eii_pwl').subs(Txx,'Txx').subs(Tyy,'Tyy').subs(Txy,'Txy'))
ddgdTii = fdg.diff(Tii).subs(fdg,'fdg').subs(dgro,'dgro').subs(dred,'dred').subs(Wpwl,'Wred').subs(exy_pwl,'exy_pwl').subs(exx_pwl,'exx_pwl').subs(eyy_pwl,'eyy_pwl').subs(Eii_pwl,'Eii_pwl').subs(Txx,'Txx').subs(Tyy,'Tyy').subs(Txy,'Txy')
print('ddgdTii = ' + octave_code(ddgdTii) + ';')

# ddgdTii1     = solve(f,ddgdTii)[0]
# ddgdTii_subs = ddgdTii1.subs(Wpwl1.diff(Tii),'dWreddTii').subs(Wpwl1,'Wred')
# display(ddgdTii_subs)
# print('ddgdTii = ' + octave_code(ddgdTii_subs) + ';')

-dt*(-2*Ared*Wred*dg + dgro*(1 - p)/dg) + 1

-Ared*dg**2*dt*((Txx + Tyy)*(-exx_pwl*n_pwl/Tii - eyy_pwl*n_pwl/Tii) + (-exx_pwl - eyy_pwl)*(Gxx/(Eii*f_ani) + Gyy/(Eii*f_ani)) - Gxx*exx_pwl*n_pwl/(Eii*f_ani) - Gxx*exx_pwl/(Eii*f_ani) - Gxy*exy_pwl*n_pwl*two/(Eii*f_ani) - Gxy*exy_pwl*two/(Eii*f_ani) - Gyy*eyy_pwl*n_pwl/(Eii*f_ani) - Gyy*eyy_pwl/(Eii*f_ani))

ddgdTii = -Ared.*dg.^2.*dt.*((Txx + Tyy).*(-exx_pwl.*n_pwl./Tii - eyy_pwl.*n_pwl./Tii) + (-exx_pwl - eyy_pwl).*(Gxx./(Eii.*f_ani) + Gyy./(Eii.*f_ani)) - Gxx.*exx_pwl.*n_pwl./(Eii.*f_ani) - Gxx.*exx_pwl./(Eii.*f_ani) - Gxy.*exy_pwl.*n_pwl.*two./(Eii.*f_ani) - Gxy.*exy_pwl.*two./(Eii.*f_ani) - Gyy.*eyy_pwl.*n_pwl./(Eii.*f_ani) - Gyy.*eyy_pwl./(Eii.*f_ani));
